In [20]:
### movement descriptor
## Hockey Training

# Import Library
import cv2
import numpy as np
import os
import math
from PIL import Image
import csv 
import pickle

In [2]:
# Directory of hockey dataset
#_path = r'C:\Users\Jinhe\Desktop\HockeyFights'
_path = r'C:\Users\Jinhe\Desktop\temp'
videos = os.listdir(_path)

In [3]:
# resize frame for unified features count
height = 720
width = 1280
# height = row
# sub_width * sub_height = number of features about movement descriptor
sub_height = int(width/15)
sub_width = int(width/15)

In [13]:
isFight = False

In [ ]:
# calculate video flows

for video in videos:
    
    if video[:1] == 'f':
        isFight = True
    else:
        isFight = False
    
    # Read video
    cap = cv2.VideoCapture(_path + '\\' + video)
    
    # init freame size
    frame_size = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = np.zeros((frame_size, height, width))
    
    for i in range(frame_size):
        (ret, frame) = cap.read()
        
        frame = np.array(Image.fromarray(frame).resize((width, height)))
        frames[i] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate flow
    flows = np.zeros((frame_size - 1, height, width, 2))
    for i in range(frame_size - 1):
        flows[i] = cv2.calcOpticalFlowFarneback(frames[i], frames[i+1], None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # Calculate Magnititude & angle
    mags = np.zeros((frame_size - 1, height, width))
    angs = np.zeros((frame_size - 1, height, width))
    
    # File open
    f = open((video[:len(video)-4] + '.csv'), 'w', encoding='utf-8', newline='')
    wr = csv.writer(f)
    wr.writerow(['isFight', 'mags', 'angs'])
    
    for i in range(frame_size - 1):
        mags[i], angs[i] = cv2.cartToPolar(flows[i, :, :, 0], flows[i, :, :, 1])
    
    dat = [mags, angs]
    
    with open (video[:len(video)-4] + '.dat', 'wb') as f:
        pickle.dump(dat, f)

In [30]:
with open ('fi10_xvid.dat', 'rb') as f:
    data = pickle.load(f)

(40, 720, 1280)

In [ ]:
# Variation of two frames that stick together
Ds = np.zeros(frame_size - 1, sub_height, sub_width)

# sub Area
sub_mags = np.zeros((frame_size - 1, 15, 15))
sub_angs = np.zeros((frame_size - 1, 15, 15))

In [ ]:

##최종적으로 각 컷마다 Motion Binary Map 저장
MBM_means = np.zeros((row_sub,col_sub,1000))
#ex) MBM_means[:,:,0] : 첫번째 컷의 움직임 서술자
    

list_len = 41 #프레임 갯수


for video in range(1000):
    
    if index % 50 == 0:
        print(index)
    
    #video번째 컷이 들어있는 파일의 경로
    path = path_+'/'+os.listdir(path_)[video]
    
    #video번째 영상 불러오기
    cap = cv2.VideoCapture(path)
    
    #해당 영상의 프레임들 다 모으기
    frames = np.zeros((row_len,col_len,list_len))

    for i in range(41):
        ret, frame = cap.read() #프레임 하나씩
       
    #flow에 따른 mag, ang (40장 : 720x1280x40)
    mags = np.zeros((row_len,col_len,list_len-1))
    angs = np.zeros((row_len,col_len,list_len-1))

    for i in range(list_len-1):
        mags[:,:,i], angs[:,:,i] = cv2.cartToPolar(flows[:,:,0,i], flows[:,:,1,i])

    #이전 프레임과의 크기 및 방향의 변화량 : Ds (48x85x39)
    Ds = np.zeros((row_sub,col_sub,list_len-2))

    #15x15인 서브영역
    sub_mags = np.zeros((15,15,list_len-1))
    sub_angs = np.zeros((15,15,list_len-1))
    
    
    # 이거 계산하는거까지 구현하기
    
    #이전 프레임과의 크기 및 방향의 변화량
    w = 0.4 #weight

    for a in range(row_sub):
        for b in range(col_sub):
            for i in range(list_len-1):
                sub_mags[:,:,i] = mags[15*a:15*(a+1),15*b:15*(b+1),i] 
                sub_angs[:,:,i] = angs[15*a:15*(a+1),15*b:15*(b+1),i]

            M = sub_mags[7,7,:]
            V = sub_angs[7,7,:]
            
            for j in range(len(M)-1):
                Ds[a,b,j] = (w*abs(M[j+1]-M[j]))+((1-w)*math.acos(V[j+1]*V[j]/abs(V[j+1]*V[j])))/math.radians(180) 
            Ds[np.isnan(Ds)]=0    


    #Motion Binary Map : 움직임 변화가 큰 부분 추출
    
    th = 2.5 #Ds가 th보다 큰 부분만

    MBM = np.zeros_like(Ds)
    MBM[Ds>th]=1
    
    #한 영상 단위 평균
    MBM_mean = MBM.mean(axis=2)

    #모든 천개의 영상 MBM_mean 저장
    MBM_means[:,:,video] = MBM_mean


In [ ]:
#엑셀에 저장
f = open('HockeyTraining.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
for i in range(1000):    
    li = list(MBM_means[:,:,i].reshape(MBM_means.shape[0]*MBM_means.shape[1]))    
    wr.writerow(li)
f.close()

##나중에 엑셀에서 위에서 500개 폭력, 아래 500개 비폭력 셀 추가